# 使用 LoRA 和 Hugging Face 进行高效的大型语言模型训练

在这个 sagemaker 示例中，我们将学习如何应用 [Low-Rank Adaptation of Large Language Models (LoRA)](https://arxiv.org/abs/2106.09685) 来微调 BLOOMZ (BLOOM 70亿参数版本指令调优版本) 在单块GPU上。 我们将利用 Hugging Face [Transformers](https://huggingface.co/docs/transformers/index), [Accelerate](https://huggingface.co/docs/accelerate/index), 以及 [PEFT](https://github.com/huggingface/peft). 

你将学到如何：

1. 设置开发环境
2. 加载并准备数据集
3. 在 Amazon SageMaker 上使用 LoRA 和 bnb int-8 来微调 BLOOM
4. 将模型部署到 Amazon SageMaker 端点

### 简介：PEFT 或参数高效微调

[PEFT](https://github.com/huggingface/peft), 或 Parameter Efficient Fine-tuning，是 Hugging Face 的一个新的开源库，可以使预训练语言模型 (PLM) 高效适应各种下游应用程序，而无需微调模型的所有参数。 PEFT 目前包括以下技术：
- LoRA: [LORA: LOW-RANK ADAPTATION OF LARGE LANGUAGE MODELS](https://arxiv.org/pdf/2106.09685.pdf)
- Prefix Tuning: [P-Tuning v2: Prompt Tuning Can Be Comparable to Fine-tuning Universally Across Scales and Tasks](https://arxiv.org/pdf/2110.07602.pdf)
- P-Tuning: [GPT Understands, Too](https://arxiv.org/pdf/2103.10385.pdf)
- Prompt Tuning: [The Power of Scale for Parameter-Efficient Prompt Tuning](https://arxiv.org/pdf/2104.08691.pdf)


In [ ]:
!pip install "transformers==4.26.0" "datasets[s3]==2.9.0" sagemaker py7zr --upgrade --quiet

如果您要在本地环境中使用 Sagemaker。 您需要访问具有 Sagemaker 所需权限的 IAM 角色。 您可以在 [此处](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) 找到更多相关信息。

In [ ]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")


## 2. 加载并准备数据集

我们将使用[samsum](https://huggingface.co/datasets/samsum)数据集，这是一个包含大约16k个带有摘要的类似信使的对话的集合。对话是由精通英语的语言学家创建并记录下来的。

```python
{
  "id": "13818513",
  "summary": "Amanda baked cookies and will bring Jerry some tomorrow.",
  "dialogue": "Amanda: I baked cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)"
}
```

要加载 `samsum` 数据集，我们使用 🤗Hugging Face Datasets 库中的 `load_dataset()` 方法。

In [ ]:
from datasets import load_dataset

# Load dataset from the hub
dataset = load_dataset("samsum", split="train")

print(f"Train dataset size: {len(dataset)}")
# Train dataset size: 14732

为了训练我们的模型，我们需要将我们的输入（文本）转换为token ID。 这是由 🤗 Transformers Tokenizer 完成的。 如果您不确定这是什么意思，请查看抱抱脸课程的**[第 6 章](https://huggingface.co/course/chapter6/1?fw=tf)**。

In [ ]:
from transformers import AutoTokenizer

model_id="bigscience/bloomz-7b1"

# Load tokenizer of BLOOMZ
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.model_max_length = 2048 # overwrite wrong value

在开始训练之前，我们需要预处理我们的数据。抽象摘要是一项文本生成任务。我们的模型将以文本作为输入，并生成摘要作为输出。我们想了解输入和输出需要多长时间才能有效地批处理数据。

我们定义了一个' prompt_template '，我们将使用它来构造一个指示提示符，以提高模型的性能。我们的' prompt_template '有一个固定的开始和结束，我们的文档在中间。这意味着我们需要确保“固定”模板部件+文档不超过模型的最大长度。
我们在训练前对数据集进行预处理，并将其保存到磁盘上，然后上传到S3。你可以在你的本地机器或CPU上运行这个步骤，并将它上传到[Hugging Face Hub](https://huggingface.co/docs/hub/datasets-overview)。

In [ ]:
from random import randint
from itertools import chain
from functools import partial

# custom instruct prompt start
prompt_template = f"Summarize the chat dialogue:\n{{dialogue}}\n---\nSummary:\n{{summary}}{{eos_token}}"

# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = prompt_template.format(dialogue=sample["dialogue"],
                                            summary=sample["summary"],
                                            eos_token=tokenizer.eos_token)
    return sample


# apply prompt template per sample
dataset = dataset.map(template_dataset, remove_columns=list(dataset.features))

print(dataset[randint(0, len(dataset))]["text"])

# empty list to save remainder from batches to use in next batch
remainder = {"input_ids": [], "attention_mask": []}


def chunk(sample, chunk_length=2048):
    # define global remainder variable to save remainder from batches to use in next batch
    global remainder
    # Concatenate all texts and add remainder from previous batch
    concatenated_examples = {k: list(chain(*sample[k])) for k in sample.keys()}
    concatenated_examples = {k: remainder[k] + concatenated_examples[k] for k in concatenated_examples.keys()}
    # get total number of tokens for batch
    batch_total_length = len(concatenated_examples[list(sample.keys())[0]])

    # get max number of chunks for batch
    if batch_total_length >= chunk_length:
        batch_chunk_length = (batch_total_length // chunk_length) * chunk_length

    # Split by chunks of max_len.
    result = {
        k: [t[i : i + chunk_length] for i in range(0, batch_chunk_length, chunk_length)]
        for k, t in concatenated_examples.items()
    }
    # add remainder to global variable for next batch
    remainder = {k: concatenated_examples[k][batch_chunk_length:] for k in concatenated_examples.keys()}
    # prepare labels
    result["labels"] = result["input_ids"].copy()
    return result


# tokenize and chunk dataset
lm_dataset = dataset.map(
    lambda sample: tokenizer(sample["text"]), batched=True, remove_columns=list(dataset.features)
).map(
    partial(chunk, chunk_length=2048),
    batched=True,
)

# Print total number of samples
print(f"Total number of samples: {len(lm_dataset)}")

处理完数据集后，我们将使用新的[文件系统整合](https://huggingface.co/docs/datasets/filesystems)将我们的数据集上传到S3。我们正在使用' sess.default_bucket() '，如果您想将数据集存储在不同的S3桶中，请调整此设置。我们将在后面的训练脚本中使用S3路径。

In [ ]:
# save train_dataset to s3
training_input_path = f's3://{sess.default_bucket()}/processed/samsum-sagemaker/train'
lm_dataset.save_to_disk(training_input_path)

print("uploaded data to:")
print(f"training dataset to: {training_input_path}")

## 3. 在 Amazon SageMaker 上使用 LoRA 和 bnb int-8 微调 BLOOM

除了 LoRA 技术，我们还将使用 [bitsanbytes LLM.int8()](https://huggingface.co/blog/hf-bitsandbytes-integration) 通过Freeze方法将 LLM 量化为 int8。 这使我们能够将 BLOOMZ 所需的内存减少约 4 倍。

我们准备了一个 [run_clm.py](./scripts/run_clm.py)，它实现了使用 PEFT 来训练我们的模型。 如果您对其工作原理感兴趣，请查看 [使用 LoRA 和 Hugging Face 进行高效大型语言模型训练](https://www.philschmid.de/fine-tune-flan-t5-peft) 博客，我们在其中详细的解释了训练脚本。


为了创建 sagemaker 训练作业，我们需要一个 HuggingFace 估计器。 Estimator 处理端到端的 Amazon SageMaker 训练和部署任务。 Estimator 管理基础设施的使用。
SagMaker 负责为我们启动和管理所有必需的 ec2 实例，提供正确的 huggingface 容器，上传提供的脚本并将数据从我们的 S3 存储桶下载到位于“/opt/ml/input/data”的容器中。 然后，它通过运行开始训练工作。



In [ ]:
import time
# define Training Job Name 
job_name = f'huggingface-peft-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'

from sagemaker.huggingface import HuggingFace

# hyperparameters, which are passed into the training job
hyperparameters ={
  'model_id': model_id,                                # pre-trained model
  'dataset_path': '/opt/ml/input/data/training', # path where sagemaker will save training dataset
  'epochs': 3,                                         # number of training epochs
  'per_device_train_batch_size': 1,                    # batch size for training
  'lr': 2e-4,                                          # learning rate used during training
}

# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'run_clm.py',      # train script
    source_dir           = 'scripts',         # directory which includes all the files needed for training
    instance_type        = 'ml.g5.2xlarge', # instances type used for the training job
    instance_count       = 1,                 # the number of instances used for training
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    volume_size          = 300,               # the size of the EBS volume in GB
    transformers_version = '4.26',            # the transformers version used in the training job
    pytorch_version      = '1.13',            # the pytorch_version version used in the training job
    py_version           = 'py39',            # the python version used in the training job
    hyperparameters      =  hyperparameters
)

我们现在可以开始我们的训练工作，使用 .fit() 方法将我们的 S3 路径传递给训练脚本。

In [ ]:
# define a data input dictonary with our uploaded s3 uris
data = {'training': training_input_path}

# starting the train job with our uploaded datasets as input
huggingface_estimator.fit(data, wait=True)

训练用了“20632”秒，大约是“5.7”小时。 我们使用的“ml.g5.2xlarge”实例每小时收费“1.515 美元”。 因此，训练 BLOOMZ 7B 的总成本为 8.63 美元。 我们可以通过使用 spot 实例来降低成本，但是通过等待或重启可能会增加训练时间。

## 4. 将模型部署到 Amazon SageMaker 端点

当使用 `peft` 进行训练时，您通常会得到适配器权重。 我们添加了 `merge_and_unload()` 方法来将基础模型与 adatper 合并，以便更轻松地部署模型。 因此我们现在可以使用 transformers 库的 pipelines 功能。

我们现在可以在我们的 HuggingFace 估计器对象上使用“deploy()”来部署我们的模型，传递我们想要的实例数量和实例类型。


In [ ]:
from sagemaker.huggingface import HuggingFaceModel

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data=huggingface_estimator.model_data,
   #model_data="s3://hf-sagemaker-inference/model.tar.gz",  # Change to your model path
   role=role, 
   transformers_version="4.26", 
   pytorch_version="1.13", 
   py_version="py39",
   model_server_workers=1
)
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type= "ml.g5.4xlarge"
)


SageMaker 通过创建 SageMaker 端点配置和 SageMaker 端点来启动部署过程。 端点配置定义模型和实例类型。

让我们使用“测试”拆分中的示例进行测试。

In [ ]:
from random import randint
from datasets import load_dataset

# Load dataset from the hub
test_dataset = load_dataset("samsum", split="test")

# select a random test sample
sample = test_dataset[randint(0,len(test_dataset))]

# format sample
prompt_template = f"Summarize the chat dialogue:\n{{dialogue}}\n---\nSummary:\n"

fomatted_sample = {
  "inputs": prompt_template.format(dialogue=sample["dialogue"]),
  "parameters": {
    "do_sample": True,
    "top_p": 0.9,
    "temperature": 0.1,
    "max_new_tokens": 100,
  }
}

# predict
res = predictor.predict(fomatted_sample)

print(res[0]["generated_text"].split("Summary:")[-1])
# Kirsten and Alex are going bowling this Friday at 7 pm. They will meet up and then go together.


让我们将其与测试结果进行比较

In [ ]:
print(sample["summary"])
# Kirsten reminds Alex that the youth group meets this Friday at 7 pm and go bowling.

最后，我们再次删除端点。

In [ ]:

predictor.delete_model()
predictor.delete_endpoint()